In [1]:
import ctypes
import gc

In [2]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [3]:
def find_object_in_gc(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return f"Found: {obj}"
    return "Not found"

In [4]:
# define class that causes circular references
class A:
    def __init__(self):
        self.b = B(self)
        print(f'A: {id(self)} - B: {id(self.b)}')

class B:
    def __init__(self, a):
        self.a = a
        print(f'B: {id(self)} - A: {id(self.a)}')

In [5]:
# disable GC
gc.disable()

In [6]:
my_var = A()

B: 140215101524792 - A: 140215101523784
A: 140215101523784 - B: 140215101524792


In [7]:
# notice B is constructed first because it is construcuted during the __init__ of A
# and now, my_var refers to an object type A, and this object refer to an object type B, and this type B object refers back to the type A object
# this is a circular reference

In [8]:
# my_var refer to type A object
print(id(my_var))

140215101523784


In [9]:
print(id(my_var.b))

140215101524792


In [10]:
print(id(my_var.b.a))

140215101523784


In [11]:
# now let store the memory addresses of these object so that we can refer to it later
b_id = id(my_var.b)
a_id = id(my_var.b.a)
print(a_id, b_id)

140215101523784 140215101524792


In [12]:
ref_count(a_id)

2

In [13]:
ref_count(b_id)

1

In [14]:
find_object_in_gc(a_id)

&#39;Found: &lt;__main__.A object at 0x7f865f518748&gt;&#39;

In [15]:
find_object_in_gc(b_id)

&#39;Found: &lt;__main__.B object at 0x7f865f518b38&gt;&#39;

In [16]:
# now, set my_var to None, that mean nothing is refer to both object A and B
my_var = None

In [17]:
ref_count(a_id)

5

In [18]:
ref_count(b_id)

1

In [19]:
# now run GC
gc.collect()

4768

In [20]:
find_object_in_gc(a_id)

&#39;Found: VersionInfo(&lt;TreeInstance of &lt;ClassValue: &lt;Class: _version_info@122-128&gt;&gt;(&lt;ValuesArguments: []&gt;)&gt;)&#39;

In [21]:
find_object_in_gc(b_id)

&#39;Not found&#39;

In [22]:
# as we see, after the GC collected, the memory addresses refer to something else or nothing at all